In [36]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

#cross entropy between two logits with temperature applied
#returns a 1d array with loss for each row in the inputs
def cross_entropy2(tar_soft_t, pred_logits):
   pred_soft_t = tf.nn.softmax(pred_logits/temprature)
#    tar_soft = tf.nn.softmax(tar_logits_t/temprature)
   pred_log = -1 * tf.log(pred_soft_t)
   product =tf.multiply(tar_soft_t,pred_log)
#    print("Swapnil", pred_log,tar_soft_t,product)
   return tf.reduce_mean(product)
    
def custom_loss(y_true, pred_logits, tar_soft_t):
#    print(tf.losses.sparse_softmax_cross_entropy(labels = y_true, logits = pred_logits))
#    print(l * cross_entropy2(tar_soft_t, pred_logits_t))
#    return loss_weight * tf.losses.sparse_softmax_cross_entropy(labels = y_true, logits = pred_logits) + (1-loss_weight) * cross_entropy2(tar_soft_t, pred_logits_t)
  return  loss_weight * tf.losses.sparse_softmax_cross_entropy(labels = y_true, logits = pred_logits)+(1-loss_weight)*cross_entropy2(tar_soft_t,pred_logits)
#     one_hot=tf.one_hot(y_true,10)
#     sess2=tf.InteractiveSession()
#     tf.train.start_queue_runners(sess2)
#     print("S_",tar_soft_t.eval())
#     print("T_",pred_soft_t.eval())
#     sess2.close()
#     return tf.losses.mean_squared_error(labels=tar_soft_t,predictions=pred_soft_t)



In [37]:
def getFilterData(f,l):
  sess=tf.InteractiveSession()
  tf.train.start_queue_runners(sess)
  data_s=f['x'].eval()
  out_s=l.eval()
  sess.close()
  return data_s,out_s


def student_model_fn(features, labels, mode):
#   data_swap,out_swap=getFilterData(features,labels)
  # Input Layer
  # Reshape X to 4-D tensor: [batch_size, width, height, channels]
  # MNIST images are 28x28 pixels, and have one color channel
  input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])
  # Convolutional Layer #1
  # Computes 32 features using a 5x5 filter with ReLU activation.
  # Padding is added to preserve width and height.
  # Input Tensor Shape: [batch_size, 28, 28, 1]
  # Output Tensor Shape: [batch_size, 28, 28, 32]
# #   if mode == tf.estimator.ModeKeys.TRAIN:
#       eval_teacher_fn = tf.estimator.inputs.numpy_input_fn(
#           x={"x":data_swap},
#           y=out_swap,
#           batch_size=100,
#           shuffle=False)

#       eval_teacher=mnist_classifier.evaluate(input_fn=eval_teacher_fn)
#       predictions=mnist_classifier.predict(input_fn=eval_teacher_fn)
#       teacher_pred=list(predictions)
#       teacher_soft=[ p['probabilities'] for p in teacher_pred]
#       temp=np.array(teacher_soft)
      
#       teacher_soft_t=tf.convert_to_tensor(temp)


  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters = No_of_Feature_Maps,
      kernel_size=[Size_of_kernel, Size_of_kernel],
      padding="same",
      activation=tf.nn.relu)

  # Pooling Layer #1
  # First max pooling layer with a 2x2 filter and stride of 2
  # Input Tensor Shape: [batch_size, 28, 28, 32]
  # Output Tensor Shape: [batch_size, 14, 14, 32]
  pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

  # Convolutional Layer #2
  # Computes 64 features using a 5x5 filter.
  # Padding is added to preserve width and height.
  # Input Tensor Shape: [batch_size, 14, 14, 32]
  # Output Tensor Shape: [batch_size, 14, 14, 64]
  conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=No_of_Feature_Maps,
      kernel_size=[Size_of_kernel, Size_of_kernel],
      padding="same",
      activation=tf.nn.relu)

  # Pooling Layer #2
  # Second max pooling layer with a 2x2 filter and stride of 2
  # Input Tensor Shape: [batch_size, 14, 14, 64]
  # Output Tensor Shape: [batch_size, 7, 7, 64]
  pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

  # Flatten tensor into a batch of vectors
  # Input Tensor Shape: [batch_size, 7, 7, 64]
  # Output Tensor Shape: [batch_size, 7 * 7 * 64]
  pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * No_of_Feature_Maps])

  # Dense Layer
  # Densely connected layer with 1024 neurons
  # Input Tensor Shape: [batch_size, 7 * 7 * 64]
  # Output Tensor Shape: [batch_size, 1024]
  dense = tf.layers.dense(inputs=pool2_flat, units=FC_UNIT, activation=tf.nn.relu)

  # Add dropout operation; 0.6 probability that element will be kept
#   if dropout:
  dropout = tf.layers.dropout(inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

  # Logits layer
  # Input Tensor Shape: [batch_size, 1024]
  # Output Tensor Shape: [batch_size, 10]
  logits = tf.layers.dense(inputs=dropout, units=10)

  predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }
  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
  #   loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
  
    
#   student_pred=predictions['probabilities']
#   sess1=tf.InteractiveSession()
#   tf.train.start_queue_runners(sess1)
#   student_soft=student_pred.eval()
#   sess1.close()  
  if mode == tf.estimator.ModeKeys.TRAIN:
#     pred_soft_t=tf.nn.softmax(logits, name="softmax_tensor")
#     print('a',labels,'b', logits,'c', teacher_soft_t ,'d', pred_logits_t)
    loss=custom_loss(labels, logits,teacher_soft_t)
  else:
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
  
    
  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
  eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"]),
  }
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)




In [38]:
mnist = tf.contrib.learn.datasets.load_dataset("mnist")
train_data = mnist.train.images  # Returns np.array
train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
eval_data = mnist.test.images  # Returns np.array
eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)
temp_list=[10]
loss_weight_list=[0.75]

##HyperParameters
lSize_of_kernel = [2,3,4,5,6]
lNo_of_Feature_Maps = [2,4,6,8,16]
lFC_UNIT=[32,64,128,256]
# lSize_of_kernel = [2,3,4,5]
# lNo_of_Feature_Maps = [2,4,8]
# lPool_stride = [1,2,3]
# lConv_Stride = [1,2,3]
# lFC_UNIT=[64,128]

p=-1
l=-1
def getdatap():
    global p
    p += 1
    if p>len(train_labels)/100:
        p=0
    data=train_data[p*100:(p+1)*100]
    labels=train_labels[p*100:(p+1)*100]
    
    datatensor=tf.convert_to_tensor(data)
    labeltensor=tf.convert_to_tensor(labels)
    
    
    return {'x':datatensor},labeltensor

def getdata_eval():
    datatensor=tf.convert_to_tensor(eval_data)
    labeltensor=tf.convert_to_tensor(eval_labels)
    return {'x':datatensor},labeltensor


out = open('ensemble.csv', 'w')

X = []
for i in temp_list:
    for j in loss_weight_list:
        for k in lSize_of_kernel:
            for l in lNo_of_Feature_Maps:
                for m in lFC_UNIT:
                    print("THIS",k,l,m)
                    Size_of_kernel = k
                    No_of_Feature_Maps = l
                    FC_UNIT = m
                    temprature=i
                    loss_weight=j
                    # Create the Estimator
                    student_classifier = tf.estimator.Estimator(
                      model_fn=student_model_fn, model_dir="./mnist_convnet_student_ensemble_t_"+str(i)+"_w_"+str(j)+"_k_"+str(k)+"_f_"+str(l)+"_fc_"+str(m))

                    eval_input_fn = tf.estimator.inputs.numpy_input_fn(
                       x={"x": eval_data},
                       y=eval_labels,
                       num_epochs=1,
                       shuffle=False)
                    eval_results=student_classifier.predict(input_fn=eval_input_fn)
                    student_pred=list(eval_results)
                    student_classes=[ p['classes'] for p in student_pred]                    
                    X.append(student_classes)
                    

Extracting MNIST-data/train-images-idx3-ubyte.gz
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz
THIS 2 2 32
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fa954df5610>, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': './mnist_convnet_student_ensemble_t_10_w_0.75_k_2_f_2_fc_32', '_save_summary_steps': 100}


NameError: global name 'dropout' is not defined

In [ ]:
from scipy import stats

X = tf.stack(X)
sess = tf.InteractiveSession()
a = X.eval()
sess.close()

x = stats.mode(a)
x = x.mode
print(eval_labels)
print((x == eval_labels).sum()/len(eval_labels))

In [ ]:
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
  x={"x": eval_data},
  y=eval_labels,
  shuffle=False)
eval_result=student_classifier.evaluate(input_fn=eval_input_fn)
 